In [9]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# 超参数设置
latent_dim = 128
img_shape = (64, 64, 3)  # 生成图像的形状
epochs = 10000
batch_size = 64


# 生成器模型
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(4 * 4 * 512, input_dim=latent_dim))
    model.add(layers.Reshape((4, 4, 512)))

    # 使用转置卷积层逐步放大到64x64
    model.add(layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'))  # 8x8
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))  # 16x16
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))  # 32x32
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding='same'))  # 64x64
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(3, kernel_size=7, padding='same', activation='tanh'))  # 输出图像
    return model


# 判别器模型
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(256, kernel_size=5, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))  # 输出概率
    return model


# 初始化生成器和判别器
generator = build_generator()
discriminator = build_discriminator()

# 编译判别器
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# 构建生成对抗网络
z = layers.Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)

# 创建组合模型
combined = tf.keras.Model(z, validity)
combined.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')


# 生成数据
def generate_latent_points(n):
    return np.random.normal(0, 1, (n, latent_dim))


# 训练 StyleGAN
for epoch in range(epochs):
    # 训练判别器
    for _ in range(2):  # 判别器训练多次
        # 生成真实和假样本
        X_real = np.random.uniform(-1, 1, (batch_size, *img_shape))  # 随机生成真实样本
        z = generate_latent_points(batch_size)
        X_fake = generator.predict(z)

        # 标签
        y_real = np.ones((batch_size, 1))
        y_fake = np.zeros((batch_size, 1))

        # 更新判别器
        d_loss_real = discriminator.train_on_batch(X_real, y_real)
        d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)

    # 训练生成器
    z = generate_latent_points(batch_size)
    g_loss = combined.train_on_batch(z, y_real)  # 目标是让生成器生成的图像被判别器判断为真实的

    # 每1000轮打印一次损失
    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, D Loss Real: {d_loss_real}, D Loss Fake: {d_loss_fake}, G Loss: {g_loss}')


# 可视化生成的样本
def plot_generated_samples(generator, n=100):
    z = generate_latent_points(n)
    samples = generator.predict(z)
    samples = 0.5 * samples + 0.5  # 归一化到 [0, 1]

    plt.figure(figsize=(10, 10))
    for i in range(n):
        plt.subplot(10, 10, i + 1)
        plt.imshow(samples[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()


plot_generated_samples(generator)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'